##### Copyright 2020 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Download necessary libraries.
%%bash 
test -d rigl || git clone https://github.com/google-research/rigl rigl_repo && mv rigl_repo/rigl ./ 
test -d gresearch || git clone https://github.com/google-research/google-research google_research

## Parameter and FLOPs Counting for MobileNetv1 

In [ ]:
import numpy as np
import tensorflow as tf
from google_research.micronet_challenge import counting
from rigl import sparse_utils
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
tf.compat.v1.reset_default_graph()
model=tf.keras.applications.MobileNet(input_shape=(224,224,3), weights=None)
model(tf.ones((2,224,224,3)))

<tf.Tensor 'mobilenet_1.00_224/act_softmax/Softmax:0' shape=(2, 1000) dtype=float32>

In [ ]:
masked_layers = []
dw_layers = []
for layer in model.layers:
  if isinstance(layer, (tf.keras.layers.Conv2D, tf.keras.layers.Dense, tf.keras.layers.DepthwiseConv2D)):     
    masked_layers.append(layer)
    if 'conv_dw' in layer.name:
      dw_layers.append(layer)
    # print(layer.name, sparse_utils._get_kernel(layer).shape)

In [ ]:
PARAM_SIZE=32
import functools

get_stats = functools.partial(
    sparse_utils.get_stats, first_layer_name='conv1',
    last_layer_name='conv_preds', param_size=PARAM_SIZE)

def print_stats(masked_layers, default_sparsity=0.8, method='erdos_renyi',
                custom_sparsities=None, is_debug=False, width=1.):
  print('Method: %s, Sparsity: %f' % (method, default_sparsity))
  total_flops, total_param_bits, sparsity = get_stats(
      masked_layers, default_sparsity=default_sparsity, method=method,
      custom_sparsities=custom_sparsities, is_debug=is_debug, width=width)
  print('Total Flops: %.3f MFlops' % (total_flops/1e6))
  print('Total Size: %.3f Mbytes' % (total_param_bits/8e6))
  print('Real Sparsity: %.3f' % (sparsity))

### Printing sparse network stats

In [ ]:
c_sparsities = {'%s/depthwise_kernel:0' % l.name: 0. for l in dw_layers}
c_sparsities_uniform = c_sparsities.copy()
c_sparsities_uniform['conv1/kernel:0'] = 0.
# c_sparsities_uniform['conv_preds/kernel:0'] = 0.
# First layer has sparsity 0 by default.
print_stats(masked_layers, 0.75, 'erdos_renyi_kernel', c_sparsities, is_debug=False)
print_stats(masked_layers, 0.75, 'random', c_sparsities_uniform, is_debug=False)
print_stats(masked_layers, 0, 'random', is_debug=False)

Method: erdos_renyi_kernel, Sparsity: 0.750000
Total Flops: 599.144 MFlops
Total Size: 4.888 Mbytes
Real Sparsity: 0.742
Method: random, Sparsity: 0.750000
Total Flops: 330.769 MFlops
Total Size: 4.894 Mbytes
Real Sparsity: 0.742
Method: random, Sparsity: 0.000000
Total Flops: 1141.544 MFlops
Total Size: 16.864 Mbytes
Real Sparsity: 0.000


In [ ]:
print_stats(masked_layers, 0.85, 'erdos_renyi_kernel', c_sparsities, is_debug=False)
print_stats(masked_layers, 0.85, 'random', c_sparsities_uniform, is_debug=False)

Method: erdos_renyi_kernel, Sparsity: 0.850000
Total Flops: 439.152 MFlops
Total Size: 3.224 Mbytes
Real Sparsity: 0.841
Method: random, Sparsity: 0.850000
Total Flops: 222.666 MFlops
Total Size: 3.229 Mbytes
Real Sparsity: 0.841


In [ ]:
print_stats(masked_layers, 0.9, 'erdos_renyi_kernel', c_sparsities, is_debug=False)
print_stats(masked_layers, 0.9, 'random', c_sparsities_uniform, is_debug=False)

Method: erdos_renyi_kernel, Sparsity: 0.900000
Total Flops: 334.134 MFlops
Total Size: 2.392 Mbytes
Real Sparsity: 0.890
Method: random, Sparsity: 0.900000
Total Flops: 168.614 MFlops
Total Size: 2.396 Mbytes
Real Sparsity: 0.890


In [ ]:
print_stats(masked_layers, 0.95, 'erdos_renyi_kernel', c_sparsities, is_debug=False)
print_stats(masked_layers, 0.95, 'random', c_sparsities_uniform, is_debug=False)

Method: erdos_renyi_kernel, Sparsity: 0.950000
Total Flops: 205.281 MFlops
Total Size: 1.560 Mbytes
Real Sparsity: 0.940
Method: random, Sparsity: 0.950000
Total Flops: 114.563 MFlops
Total Size: 1.563 Mbytes
Real Sparsity: 0.940


## Finding the width Multiplier for small dense model

In [ ]:
_, sparse_bits, _ = get_stats(masked_layers, 0.75, 'erdos_renyi_kernel', {'conv1/kernel:0':0.})
_, bits, _ = get_stats(masked_layers, 0., 'erdos_renyi_kernel', width=0.47)
print(sparse_bits/bits)
print_stats(masked_layers, 0., 'erdos_renyi_kernel', is_debug=False, width=0.47)
print_stats(masked_layers, 0.75, 'erdos_renyi_kernel', {'conv1/kernel:0':0.}, is_debug=False, width=1)

0.9933069386323201
Method: erdos_renyi_kernel, Sparsity: 0.000000
Total Flops: 266.539 MFlops
Total Size: 4.789 Mbytes
Real Sparsity: 0.000
Method: erdos_renyi_kernel, Sparsity: 0.750000
Total Flops: 588.355 MFlops
Total Size: 4.757 Mbytes
Real Sparsity: 0.750


In [ ]:
_, sparse_bits, _ = get_stats(masked_layers, 0.85, 'erdos_renyi_kernel', {'conv1/kernel:0':0.})
_, bits, _ = get_stats(masked_layers, 0., 'erdos_renyi_kernel', width=0.353)
print(sparse_bits/bits)
print_stats(masked_layers, 0., 'erdos_renyi_kernel', is_debug=False, width=0.353)
print_stats(masked_layers, 0.85, 'erdos_renyi_kernel', {'conv1/kernel:0':0.}, is_debug=False, width=1)

0.9998127484496482
Method: erdos_renyi_kernel, Sparsity: 0.000000
Total Flops: 154.770 MFlops
Total Size: 3.076 Mbytes
Real Sparsity: 0.000
Method: erdos_renyi_kernel, Sparsity: 0.850000
Total Flops: 422.419 MFlops
Total Size: 3.075 Mbytes
Real Sparsity: 0.850


In [ ]:
_, sparse_bits, _ = get_stats(masked_layers, 0.9, 'erdos_renyi_kernel', {'conv1/kernel:0':0.})
_, bits, _ = get_stats(masked_layers, 0., 'erdos_renyi_kernel', width=0.285)
print(sparse_bits/bits)
print_stats(masked_layers, 0., 'erdos_renyi_kernel', is_debug=False, width=0.285)
print_stats(masked_layers, 0.9, 'erdos_renyi_kernel', {'conv1/kernel:0':0.}, is_debug=False, width=1)

0.9996546850118981
Method: erdos_renyi_kernel, Sparsity: 0.000000
Total Flops: 103.825 MFlops
Total Size: 2.236 Mbytes
Real Sparsity: 0.000
Method: erdos_renyi_kernel, Sparsity: 0.900000
Total Flops: 312.956 MFlops
Total Size: 2.235 Mbytes
Real Sparsity: 0.900


In [ ]:
_, sparse_bits, _ = get_stats(masked_layers, 0.95, 'erdos_renyi_kernel', {'conv1/kernel:0':0.})
_, bits, _ = get_stats(masked_layers, 0., 'erdos_renyi_kernel', width=0.204)
print(sparse_bits/bits)
print_stats(masked_layers, 0., 'erdos_renyi_kernel', is_debug=False, width=0.204)
print_stats(masked_layers, 0.95, 'erdos_renyi_kernel', {'conv1/kernel:0':0.}, is_debug=False, width=1)

0.9982463429660301
Method: erdos_renyi_kernel, Sparsity: 0.000000
Total Flops: 56.617 MFlops
Total Size: 1.396 Mbytes
Real Sparsity: 0.000
Method: erdos_renyi_kernel, Sparsity: 0.950000
Total Flops: 180.359 MFlops
Total Size: 1.393 Mbytes
Real Sparsity: 0.950


### Big-Sparse Networks

In [ ]:
# BIGGER
_, sparse_bits, _ = get_stats(masked_layers, 0., 'erdos_renyi_kernel')
_, bits, _ = get_stats(masked_layers, 0.75, 'erdos_renyi_kernel', {'conv1/kernel:0':0.}, width=1.98)
print(sparse_bits/bits)
print_stats(masked_layers, 0.75, 'erdos_renyi_kernel', {'conv1/kernel:0':0.}, is_debug=False, width=1.98)
print_stats(masked_layers, 0.75, 'random', {'conv_preds/kernel:0':0.8, 'conv1/kernel:0':0.}, is_debug=False, width=1.98)
print_stats(masked_layers, 0., 'erdos_renyi_kernel', is_debug=False, width=1)

1.0084815029856933
Method: erdos_renyi_kernel, Sparsity: 0.750000
Total Flops: 2180.140 MFlops
Total Size: 16.723 Mbytes
Real Sparsity: 0.742
Method: random, Sparsity: 0.750000
Total Flops: 1122.572 MFlops
Total Size: 15.863 Mbytes
Real Sparsity: 0.757
Method: erdos_renyi_kernel, Sparsity: 0.000000
Total Flops: 1141.544 MFlops
Total Size: 16.864 Mbytes
Real Sparsity: 0.000


In [ ]:
_, sparse_bits, _ = get_stats(masked_layers, 0., 'erdos_renyi_kernel')
_, bits, _ = get_stats(masked_layers, 0.85, 'erdos_renyi_kernel', {'conv1/kernel:0':0.}, width=2.52)
print(sparse_bits/bits)
print_stats(masked_layers, 0.85, 'erdos_renyi_kernel', {'conv1/kernel:0':0.}, is_debug=False, width=2.52)
print_stats(masked_layers, 0., 'erdos_renyi_kernel', is_debug=False, width=1)

1.0032864697591513
Method: erdos_renyi_kernel, Sparsity: 0.850000
Total Flops: 2442.726 MFlops
Total Size: 16.809 Mbytes
Real Sparsity: 0.846
Method: erdos_renyi_kernel, Sparsity: 0.000000
Total Flops: 1141.544 MFlops
Total Size: 16.864 Mbytes
Real Sparsity: 0.000


In [ ]:
_, sparse_bits, _ = get_stats(masked_layers, 0., 'erdos_renyi_kernel')
_, bits, _ = get_stats(masked_layers, 0.9, 'erdos_renyi_kernel', {'conv1/kernel:0':0.}, width=3.)
print(sparse_bits/bits)
print_stats(masked_layers, 0.9, 'erdos_renyi_kernel', {'conv1/kernel:0':0.}, is_debug=False, width=3.)
print_stats(masked_layers, 0.9, 'random', {'conv_preds/kernel:0':0.8, 'conv1/kernel:0':0.}, is_debug=False, width=3.)
print_stats(masked_layers, 0., 'erdos_renyi_kernel', is_debug=False, width=1)

1.0120353164650686
Method: erdos_renyi_kernel, Sparsity: 0.900000
Total Flops: 2452.785 MFlops
Total Size: 16.664 Mbytes
Real Sparsity: 0.899
Method: random, Sparsity: 0.900000
Total Flops: 1058.478 MFlops
Total Size: 17.833 Mbytes
Real Sparsity: 0.890
Method: erdos_renyi_kernel, Sparsity: 0.000000
Total Flops: 1141.544 MFlops
Total Size: 16.864 Mbytes
Real Sparsity: 0.000


In [ ]:
_, sparse_bits, _ = get_stats(masked_layers, 0., 'erdos_renyi_kernel')
_, bits, _ = get_stats(masked_layers, 0.95, 'erdos_renyi_kernel', {'conv1/kernel:0':0.}, width=3.98)
print(sparse_bits/bits)
print_stats(masked_layers, 0.95, 'erdos_renyi_kernel', {'conv1/kernel:0':0.}, is_debug=False, width=3.98)
print_stats(masked_layers, 0., 'erdos_renyi_kernel', is_debug=False, width=1)

1.0031304863290271
Method: erdos_renyi_kernel, Sparsity: 0.950000
Total Flops: 2132.954 MFlops
Total Size: 16.812 Mbytes
Real Sparsity: 0.954
Method: erdos_renyi_kernel, Sparsity: 0.000000
Total Flops: 1141.544 MFlops
Total Size: 16.864 Mbytes
Real Sparsity: 0.000
